In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";   

In [3]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 38.1 MB/s 
     |████████████████████████████████| 22.3 MB 2.0 MB/s 
     |████████████████████████████████| 981 kB 41.4 MB/s 
     |████████████████████████████████| 263 kB 42.6 MB/s 
     |████████████████████████████████| 2.8 MB 34.8 MB/s 
     |████████████████████████████████| 1.2 MB 38.0 MB/s 
     |████████████████████████████████| 468 kB 42.6 MB/s 
     |████████████████████████████████| 3.3 MB 27.3 MB/s 
     |████████████████████████████████| 596 kB 33.9 MB/s 
     |████████████████████████████████| 880 kB 41.6 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
  Created wheel for ktrain: filename=ktrain-0.30.0-py3-none-any.whl size=25309161 sha256=89d4c7592604b29b0e1a987688ff593441a4d039ed8763b2734e395d865ff7e9
  Stored in directory: /root/.cache/pip/wheels/25/00/ae/80d1b19dd02bc2e6bc9bac32ba7dbaa59aad789e93274e4868
  Created wheel for keras-bert: filename=keras

In [4]:
import ktrain
from ktrain import text

In [5]:
import numpy as np
train = open('data_for_train_and_test_result.txt', encoding='utf-8')

lines = train.readlines()
target = []
phrase = []


for line in lines:
  target.append(float(line[0]))
  phrase.append(line[2:])
  

target = np.array(target)
phrase = np.array(phrase)


In [6]:
total_len = len(target)

x_test = phrase[: total_len // 4]
x_train = phrase[total_len // 4 : ]

y_test = target[: total_len // 4]
y_train = target[total_len // 4 : ]

In [7]:
trn, val, preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          class_names= ['real', 'generated'],
                                          preprocess_mode='distilbert',
                                          maxlen=350)

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

preprocessing train...
language: ru
train sequence lengths:
	mean : 25
	95percentile : 53
	99percentile : 56


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:641: UserWarning: class_names implies classification but targets array contains float(s) instead of integers or strings
  warnings.warn('class_names implies classification but targets array contains float(s) instead of integers or strings')


Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: ru
test sequence lengths:
	mean : 24
	95percentile : 54
	99percentile : 56


task: text classification


In [8]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [9]:
model = text.text_classifier('distilbert', train_data=trn, preproc=preproc)

Is Multi-Label? False
maxlen is 350


Downloading:   0%|          | 0.00/911M [00:00<?, ?B/s]

done.


In [10]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [11]:
learner.fit_onecycle(3e-5, 5)



begin training using onecycle policy with max lr of 3e-05...
13/13 [==============================] - 23s 696ms/step - loss: 0.6076 - accuracy: 0.8133 - val_loss: 0.4388 - val_accuracy: 1.0000


In [12]:
p = ktrain.get_predictor(model, preproc)

In [ ]:
p.predict("Сегодня замечательная погода")

In [16]:
ktrain.get_predictor(model, preproc).save('distilbert')

In [17]:
model = ktrain.load_predictor('distilbert').model

In [19]:
p = ktrain.get_predictor(model, preproc)
p.predict("Сегодня замечательная погода")

'real'